# Part 1, Topic 1, Lab B: AES Loop Skip Fault Attack in Practice

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the last lab, we showed recovering an AES becomes trivial if we're able to skip the repeated rounds of AES. In this lab, we'll look at applying this attack to a real AES implementation.*

**This lab is only supported with TINYAES128C firmware**

**LEARNING OUTCOMES:**
* Understanding how C code can be modified by a compiler
* Using a theoretical fault model to mount a real attack

## Can we skip the loop of AES

For this particular fault, the answer is that it depends! Let's take a look at the source code for TINYAES:

```C
// Cipher is the main function that encrypts the PlainText.
static void Cipher(void)
{
  uint8_t round = 0;

  // Add the First round key to the state before starting the rounds.
  AddRoundKey(0); 
  
  // There will be Nr rounds.
  // The first Nr-1 rounds are identical.
  // These Nr-1 rounds are executed in the loop below.
  for(round = 1; round < Nr; ++round)
  {
    SubBytes();
    ShiftRows();
    MixColumns();
    AddRoundKey(round);
  }
  
  // The last round is given below.
  // The MixColumns function is not here in the last round.
  SubBytes();
  ShiftRows();
  AddRoundKey(Nr);
}
```

We learned in Fault101 that fault injection can be used to bypass the final check and stay in a loop longer. Similarly, glitching can also be used to break out of a loop early. This is likely one of the effects that we saw when glitching past a password check in that course as well. That being said, it's up to the compiler how this loop gets translated to assembly. For example, `Nr` is constant here, so the compiler might decided to completely unroll the loop, preventing us from ever breaking free! For our glitch to work we really have three requirements:

1. There actually needs to be a loop for us to break free from. The compiler can't unroll the loop.
1. The registers need to line up properly such that, if we skip the loop, the last round still completes properly.
1. The compiler can't skip the first `round < Nr` check. If the compiler doesn't know that `round` always starts off less than `Nr`, it needs to do a check at the start of the loop.

unfortunately for us, while our default build of TINYAES128C will fulfill the first two requirements, if we look at the generated assembly from `simpleserial-aes-PLATFORM.lss` (CWLITEARM in this case):

```C
// Cipher is the main function that encrypts the PlainText.
static void Cipher(void)
{
 8001378:	e92d 4ff8 	stmdb	sp!, {r3, r4, r5, r6, r7, r8, r9, sl, fp, lr}
  uint8_t round = 0;

  // Add the First round key to the state before starting the rounds.
  AddRoundKey(0); 
 800137c:	2000      	movs	r0, #0
 800137e:	f7ff ffa1 	bl	80012c4 <AddRoundKey>
  
  // There will be Nr rounds.
  // The first Nr-1 rounds are identical.
  // These Nr-1 rounds are executed in the loop below.
  for(round = 1; round < Nr; ++round)
 8001382:	2401      	movs	r4, #1
  {
    SubBytes();
 8001384:	f7ff ffb8 	bl	80012f8 <SubBytes>
    ShiftRows();
 8001388:	f7ff ffce 	bl	8001328 <ShiftRows>
  for(i = 0; i < 4; ++i)
 800138c:	4b1f      	ldr	r3, [pc, #124]	; (800140c <Cipher+0x94>)
 800138e:	f8d3 10b4 	ldr.w	r1, [r3, #180]	; 0xb4
 8001392:	f101 0b10 	add.w	fp, r1, #16
    t   = (*state)[i][0];
 8001396:	f891 a000 	ldrb.w	sl, [r1]
    Tmp = (*state)[i][0] ^ (*state)[i][1] ^ (*state)[i][2] ^ (*state)[i][3] ;
 800139a:	784e      	ldrb	r6, [r1, #1]
 800139c:	788d      	ldrb	r5, [r1, #2]
 800139e:	f891 9003 	ldrb.w	r9, [r1, #3]
 80013a2:	ea8a 0006 	eor.w	r0, sl, r6
 80013a6:	ea85 0809 	eor.w	r8, r5, r9
 80013aa:	ea88 0700 	eor.w	r7, r8, r0
    Tm  = (*state)[i][0] ^ (*state)[i][1] ; Tm = xtime(Tm);  (*state)[i][0] ^= Tm ^ Tmp ;
 80013ae:	f7ff ffd9 	bl	8001364 <xtime>
 80013b2:	ea8a 0000 	eor.w	r0, sl, r0
 80013b6:	4078      	eors	r0, r7
 80013b8:	7008      	strb	r0, [r1, #0]
    Tm  = (*state)[i][1] ^ (*state)[i][2] ; Tm = xtime(Tm);  (*state)[i][1] ^= Tm ^ Tmp ;
 80013ba:	ea86 0005 	eor.w	r0, r6, r5
 80013be:	f7ff ffd1 	bl	8001364 <xtime>
 80013c2:	4046      	eors	r6, r0
 80013c4:	407e      	eors	r6, r7
 80013c6:	704e      	strb	r6, [r1, #1]
    Tm  = (*state)[i][2] ^ (*state)[i][3] ; Tm = xtime(Tm);  (*state)[i][2] ^= Tm ^ Tmp ;
 80013c8:	4640      	mov	r0, r8
 80013ca:	f7ff ffcb 	bl	8001364 <xtime>
 80013ce:	4045      	eors	r5, r0
 80013d0:	407d      	eors	r5, r7
 80013d2:	708d      	strb	r5, [r1, #2]
    Tm  = (*state)[i][3] ^ t ;        Tm = xtime(Tm);  (*state)[i][3] ^= Tm ^ Tmp ;
 80013d4:	ea8a 0009 	eor.w	r0, sl, r9
 80013d8:	f7ff ffc4 	bl	8001364 <xtime>
 80013dc:	ea89 0900 	eor.w	r9, r9, r0
 80013e0:	ea87 0709 	eor.w	r7, r7, r9
 80013e4:	70cf      	strb	r7, [r1, #3]
  for(i = 0; i < 4; ++i)
 80013e6:	3104      	adds	r1, #4
 80013e8:	4559      	cmp	r1, fp
 80013ea:	d1d4      	bne.n	8001396 <Cipher+0x1e>
    MixColumns();
    AddRoundKey(round);
 80013ec:	4620      	mov	r0, r4
  for(round = 1; round < Nr; ++round)
 80013ee:	3401      	adds	r4, #1
 80013f0:	b2e4      	uxtb	r4, r4
    AddRoundKey(round);
 80013f2:	f7ff ff67 	bl	80012c4 <AddRoundKey>
  for(round = 1; round < Nr; ++round)
 80013f6:	2c0a      	cmp	r4, #10                ; <-- round != Nr check
 80013f8:	d1c4      	bne.n	8001384 <Cipher+0xc> ; <--
  }
  
  // The last round is given below.
  // The MixColumns function is not here in the last round.
  SubBytes();
 80013fa:	f7ff ff7d 	bl	80012f8 <SubBytes>
  ShiftRows();
 80013fe:	f7ff ff93 	bl	8001328 <ShiftRows>
  AddRoundKey(Nr);
 8001402:	4620      	mov	r0, r4
}
 8001404:	e8bd 4ff8 	ldmia.w	sp!, {r3, r4, r5, r6, r7, r8, r9, sl, fp, lr}

```
it won't fulfill the last one. This means that the device will always go through one round of AES before we have the opportunity to break out of the loop. An attack is still possible on this single round of AES (with only 2 faults to boot), but the math behind it is much more complicated. We'll look at this attack in the next lab, but it would still be good to see our simpler attack working on real hardware. Luckily, it's actually pretty easy to get the firmware to fulfill the last  requirement! All we need to do is make `round` a volatile variable:

```C
// Cipher is the main function that encrypts the PlainText.
static void Cipher(void)
{
  volatile uint8_t round = 0;

  // Add the First round key to the state before starting the rounds.
  AddRoundKey(0); 
  
  // There will be Nr rounds.
  // The first Nr-1 rounds are identical.
  // These Nr-1 rounds are executed in the loop below.
  for(round = 1; round < Nr; ++round)
  {
    SubBytes();
    ShiftRows();
    MixColumns();
    AddRoundKey(round);
  }
  
  // The last round is given below.
  // The MixColumns function is not here in the last round.
  SubBytes();
  ShiftRows();
  AddRoundKey(Nr);
}
```

This will result in the following assembly (again on CWLITEARM):

```C
// Cipher is the main function that encrypts the PlainText.
static void Cipher(void)
{
 8001378:	e92d 4ff7 	stmdb	sp!, {r0, r1, r2, r4, r5, r6, r7, r8, r9, sl, fp, lr}
  volatile uint8_t round = 0;
 800137c:	2000      	movs	r0, #0
 800137e:	f88d 0007 	strb.w	r0, [sp, #7]
    t   = (*state)[i][0];
 8001382:	4f29      	ldr	r7, [pc, #164]	; (8001428 <Cipher+0xb0>)

  // Add the First round key to the state before starting the rounds.
  AddRoundKey(0); 
 8001384:	f7ff ff9e 	bl	80012c4 <AddRoundKey>
  
  // There will be Nr rounds.
  // The first Nr-1 rounds are identical.
  // These Nr-1 rounds are executed in the loop below.
  for(round = 1; round < Nr; ++round)
 8001388:	2301      	movs	r3, #1
 800138a:	f88d 3007 	strb.w	r3, [sp, #7]
 800138e:	f89d 3007 	ldrb.w	r3, [sp, #7]
 8001392:	2b09      	cmp	r3, #9                  ; <-- round < Nr check
 8001394:	d909      	bls.n	80013aa <Cipher+0x32> ; <--
    AddRoundKey(round);
  }
  
  // The last round is given below.
  // The MixColumns function is not here in the last round.
  SubBytes();
 8001396:	f7ff ffaf 	bl	80012f8 <SubBytes>
  ShiftRows();
 800139a:	f7ff ffc5 	bl	8001328 <ShiftRows>
  AddRoundKey(Nr);
 800139e:	200a      	movs	r0, #10
}
 80013a0:	b003      	add	sp, #12
 80013a2:	e8bd 4ff0 	ldmia.w	sp!, {r4, r5, r6, r7, r8, r9, sl, fp, lr}
  AddRoundKey(Nr);
 80013a6:	f7ff bf8d 	b.w	80012c4 <AddRoundKey>
    SubBytes();
 80013aa:	f7ff ffa5 	bl	80012f8 <SubBytes>
    ShiftRows();
 80013ae:	f7ff ffbb 	bl	8001328 <ShiftRows>
  for(i = 0; i < 4; ++i)
 80013b2:	f8d7 10b4 	ldr.w	r1, [r7, #180]	; 0xb4
 80013b6:	f101 0a10 	add.w	sl, r1, #16
    t   = (*state)[i][0];
 80013ba:	f891 9000 	ldrb.w	r9, [r1]
    Tmp = (*state)[i][0] ^ (*state)[i][1] ^ (*state)[i][2] ^ (*state)[i][3] ;
 80013be:	784d      	ldrb	r5, [r1, #1]
 80013c0:	788c      	ldrb	r4, [r1, #2]
 80013c2:	f891 8003 	ldrb.w	r8, [r1, #3]
 80013c6:	ea89 0005 	eor.w	r0, r9, r5
 80013ca:	ea84 0b08 	eor.w	fp, r4, r8
 80013ce:	ea8b 0600 	eor.w	r6, fp, r0
    Tm  = (*state)[i][0] ^ (*state)[i][1] ; Tm = xtime(Tm);  (*state)[i][0] ^= Tm ^ Tmp ;
 80013d2:	f7ff ffc7 	bl	8001364 <xtime>
 80013d6:	ea89 0000 	eor.w	r0, r9, r0
 80013da:	4070      	eors	r0, r6
 80013dc:	7008      	strb	r0, [r1, #0]
    Tm  = (*state)[i][1] ^ (*state)[i][2] ; Tm = xtime(Tm);  (*state)[i][1] ^= Tm ^ Tmp ;
 80013de:	ea85 0004 	eor.w	r0, r5, r4
 80013e2:	f7ff ffbf 	bl	8001364 <xtime>
 80013e6:	4045      	eors	r5, r0
 80013e8:	4075      	eors	r5, r6
 80013ea:	704d      	strb	r5, [r1, #1]
    Tm  = (*state)[i][2] ^ (*state)[i][3] ; Tm = xtime(Tm);  (*state)[i][2] ^= Tm ^ Tmp ;
 80013ec:	4658      	mov	r0, fp
 80013ee:	f7ff ffb9 	bl	8001364 <xtime>
 80013f2:	4044      	eors	r4, r0
 80013f4:	4074      	eors	r4, r6
 80013f6:	708c      	strb	r4, [r1, #2]
    Tm  = (*state)[i][3] ^ t ;        Tm = xtime(Tm);  (*state)[i][3] ^= Tm ^ Tmp ;
 80013f8:	ea89 0008 	eor.w	r0, r9, r8
 80013fc:	f7ff ffb2 	bl	8001364 <xtime>
 8001400:	ea88 0800 	eor.w	r8, r8, r0
 8001404:	ea86 0608 	eor.w	r6, r6, r8
 8001408:	70ce      	strb	r6, [r1, #3]
  for(i = 0; i < 4; ++i)
 800140a:	3104      	adds	r1, #4
 800140c:	4551      	cmp	r1, sl
 800140e:	d1d4      	bne.n	80013ba <Cipher+0x42>
    AddRoundKey(round);
 8001410:	f89d 0007 	ldrb.w	r0, [sp, #7]
 8001414:	f7ff ff56 	bl	80012c4 <AddRoundKey>
  for(round = 1; round < Nr; ++round)
 8001418:	f89d 3007 	ldrb.w	r3, [sp, #7]
 800141c:	3301      	adds	r3, #1
 800141e:	b2db      	uxtb	r3, r3
 8001420:	f88d 3007 	strb.w	r3, [sp, #7]
 8001424:	e7b3      	b.n	800138e <Cipher+0x16>
 8001426:	bf00      	nop
 8001428:	200006dc 	.word	0x200006dc
```

This time, we can see the check is being done at the beginning of the loop instead of at the end at address 0x8001392. If you look closely, you can see if we skip this check, we go directly into the last round! Make this change in your own `hardware/victims/firmware/crypto/TINYAES128C/aes.c` and we can start the lab!

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER='SS_VER_2_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
fw_path = "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

## Getting a Fault

There's two problems that we're going to face in getting a fault out of this code:

1. Where do we insert the fault? Obviously, it'll be somewhere near the beginning, but it would be nice to narrow down the location
1. How do we know we've gotten the specific fault we're looking for? Inserting faults nearby to the correct location will generate faults in the output, but it's hard to tell just from looking at the output if we've broken out of the loop.

We can pretty easily solve both these problems with power analysis! Since the different operations of AES are pretty distinct in AES, we can visually inspect an unfaulted power trace to know where to insert the fault. For the second issue, we can again visually inspect the power trace. Breaking out of the loop will look very different to completing the rest of AES.

Let's get the target to encrypt something and capture a power trace. We'll also capture a copy of the ciphertext, which we can use to detect glitches in general:

In [ ]:
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.arm()
target.simpleserial_write('p', bytearray([0]*16))
ret = scope.capture()
if ret:
    print("No trigger!")

wave = scope.get_last_trace()

output = target.simpleserial_read_witherrors('r', 16)
gold_ct = output['payload']

print(gold_ct)

In [ ]:
cw.plot(wave[:2000])

Now select a range of values of where to glitch. Remember the add round key operation is done at the beginning, then at the end of very loop through an AES round.  You should be glitching between the last little bit of this operation and the beginning of the next one.

In [ ]:
glitch_loc = range(300, 340)

By this point, you should have some pretty reliable settings for glitching the target, so the provided glitch loop won't even use the GlitchController, but feel free to add it in if you're not super confident in using only one pair of glitch settings.

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.io.hs2 = "glitch"
# These width/offset numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1:
scope.glitch.width = 3
scope.glitch.offset = -12.8
print(scope.glitch)

You could use a SAD comparison to detect when the glitch looks substantially different, but it'll be a bit easier to loop until we glitch, then check what the waveform looks like. If you get a glitch, but it doesn't result in the correct effect, you can pretty safely skip that glitch spot.

In [ ]:
from tqdm.notebook import tqdm, trange
wave = None
import logging
ktp = cw.ktp.Basic()
logging.getLogger().setLevel(logging.ERROR)
reboot_flush()
for i in trange(min(glitch_loc), max(glitch_loc) + 1):
    scope.adc.timeout = 0.2
    scope.glitch.ext_offset = i
    ack = None
    while ack is None:
        target.simpleserial_write('k', ktp.next()[0])
        ack = target.simpleserial_wait_ack()
        if ack is None:
            reboot_flush()
            time.sleep(0.1)
    
    scope.arm()
    
    pt = bytearray([0]*16)
    target.simpleserial_write('p', pt)
    ret = scope.capture()
    if ret:
        reboot_flush() #bad if we accidentally didn't have this work
        time.sleep(0.1)
        print("timed out!")
        continue
    output = target.simpleserial_read_witherrors('r', 16, glitch_timeout = 1)
    if output['valid']:
        if output['payload'] != gold_ct:
            print("Glitched at {}".format(i))
            wave = scope.get_last_trace()
            break
    else:
        reboot_flush()
        
cw.plot(wave)

Let's collect our glitched ciphertext:

In [ ]:
glitched_ct0 = bytearray(output['payload'])

In [ ]:
glitched_ct0

In [ ]:
pt0 = bytearray(pt)

We can repeat this twice more with different plaintexts to get all the faults needed:

In [ ]:
from tqdm.notebook import tqdm, trange
wave = None
import logging
ktp = cw.ktp.Basic()
logging.getLogger().setLevel(logging.ERROR)
reboot_flush()
while True:
    scope.adc.timeout = 0.2
    scope.glitch.ext_offset = i #should still be in the right spot from the last glitch
    ack = None
    while ack is None:
        target.simpleserial_write('k', ktp.next()[0])
        ack = target.simpleserial_wait_ack()
        if ack is None:
            reboot_flush()
            time.sleep(0.1)
    
    scope.arm()
    
    pt = bytearray([1]*16)
    target.simpleserial_write('p', pt)
    ret = scope.capture()
    if ret:
        reboot_flush() #bad if we accidentally didn't have this work
        time.sleep(0.1)
        print("timed out!")
        continue
    output = target.simpleserial_read_witherrors('r', 16, glitch_timeout = 1)
    if output['valid']:
        if output['payload'] != gold_ct:
            print("Glitched at {}".format(i))
            wave = scope.get_last_trace()
            break
    else:
        reboot_flush()
        
cw.plot(wave)

In [ ]:
glitched_ct1 = bytearray(output['payload'])
pt1 = bytearray(pt)

In [ ]:
from tqdm.notebook import tqdm, trange
wave = None
import logging
ktp = cw.ktp.Basic()
logging.getLogger().setLevel(logging.ERROR)
reboot_flush()
while True:
    scope.adc.timeout = 0.2
    scope.glitch.ext_offset = i #should still be in the right spot from the last glitch
    ack = None
    while ack is None:
        target.simpleserial_write('k', ktp.next()[0])
        ack = target.simpleserial_wait_ack()
        if ack is None:
            reboot_flush()
            time.sleep(0.1)
    
    scope.arm()
    
    pt = bytearray([2]*16)
    target.simpleserial_write('p', pt)
    ret = scope.capture()
    if ret:
        reboot_flush() #bad if we accidentally didn't have this work
        time.sleep(0.1)
        print("timed out!")
        continue
    output = target.simpleserial_read_witherrors('r', 16, glitch_timeout = 1)
    if output['valid']:
        if output['payload'] != gold_ct:
            print("Glitched at {}".format(i))
            wave = scope.get_last_trace()
            break
    else:
        reboot_flush()
        
cw.plot(wave)

In [ ]:
glitched_ct2 = bytearray(output['payload'])
pt2 = bytearray(pt)

In [ ]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

In [ ]:
key_guess = []
for kbyte in range(16):
    for i in range(255):
        if (sbox[i ^ pt0[kbyte]] ^ sbox[i ^ pt1[kbyte]]) == (glitched_ct0[kbyte] ^ glitched_ct1[kbyte]):
            if (sbox[i ^ pt0[kbyte]] ^ sbox[i ^ pt2[kbyte]]) == (glitched_ct0[kbyte] ^ glitched_ct2[kbyte]):
                print(i)
                key_guess.append(i)

In [ ]:
bytearray(key_guess)

You'll see that you've got the correct key bytes, but they're out of order! This is because we didn't account for the ShiftRows operation (this doesn't matter for the analysis since we were using repeated bytes for the plaintext. If this wasn't the case, we'd have to match the plaintext up with the ciphertext instead of just leaving it to the end). If we swap the bytes of the key around based on ShiftRows:

In [ ]:
SR = [0, 13, 10, 7, 4, 1, 14, 11, 8, 5, 2, 15, 12, 9, 6, 3]
for i in range(16):
    print(hex(key_guess[SR[i]]))

We can print the key in the correct order.

## Conclusions & Next Steps

Though we didn't glitch an unmodified version of TINYAES128C, we did see that, given the right implementation of AES, we can use glitching to skip the repeated rounds of AES, thus bypassing its security and allowing us to recover the key. In the next lab, we'll look at mounting an attack against an unmodified TINYAES.

---
<small>NO-FUN DISCLAIMER: This material is Copyright (C) NewAE Technology Inc., 2015-2020. ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

Tutorials derived from our open-source work must be released under the associated open-source license, and notice of the source must be *clearly displayed*. Only original copyright holders may license or authorize other distribution - while NewAE Technology Inc. holds the copyright for many tutorials, the github repository includes community contributions which we cannot license under special terms and **must** be maintained as an open-source release. Please contact us for special permissions (where possible).

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</small>